In [1]:
import os,sys,struct
import tifffile
import numpy as np
import json, glob
import datajoint as dj
from datetime import datetime
import pandas as pd
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
from brain_atlas_toolkit import graph_tools
import matplotlib.pyplot as plt
%matplotlib inline

## Read in all neurons

In [5]:
neuron_dir = '/home/ahoag/progs/mouselight/public/json30'
json_files = sorted(glob.glob(neuron_dir + '/*json'))

## Write all neuron skeletons to same directory
This will reduce the number of layers created when someone wants to view multiple neurons. The only thing we'll have to worry about here is making it clear what integer ids correspond to what neurons. Can just use 0,1,2 for soma, axon, dendrite of first neuron, 3,4,5 for soma, axon and dendrite of second neuron and so on. Use segment properties to keep all of this straight. 

In [6]:
def write_skeletons_singledir(neuron,ID_soma, debug=False):
    """ Given a neuron dictionary and the segment ID for its soma,
    create skeletons for the soma, axon and dendrite and save them in the 
    public Princeton neuroglancer bucket"""
    
    # Make directory if it does not exist already
    precomputed_dir = "/jukebox/LightSheetData/neuroglancer/public/mouselight/allen_ccfv3_25um/ccfv3_25um_mouselight"
    
    ### Soma
    coordinates_soma = []
    soma = neuron['soma']
    
    # Downsample to 25 micron / voxels and swap axes to conform with our Allen Atlas
    newx = soma['y']/10/2.5
    newy = soma['x']/10/2.5
    newz = soma['z']/10/2.5 
    coordinates_soma= [[newx,newy,newz]]
    vertex_positions_soma = np.array(coordinates_soma,dtype='<f4')
    edges_soma = np.array([],dtype='<u4')
    
    filename_soma = os.path.join(precomputed_dir,str(ID_soma))
    with open(filename_soma,'wb') as outfile:
        buf = struct.pack('<II', vertex_positions_soma.shape[0], edges_soma.shape[0])
        buf += vertex_positions_soma.tobytes()
        buf += edges_soma.tobytes()
        outfile.write(buf)
    if debug:
        print("Wrote ",filename_soma)
    
    ### Axon
    ID_axon = ID_soma + 1
    axon = neuron['axon']
    
    coordinates_axon = []
    edges_axon = []
    axon_branches_dict = {}
    axon_terminals_dict = {}
    for pt_index,pt in enumerate(axon):
       # Downsample to to 25 micron / voxels and swap their axes
        newx = pt['y']/10/2.5
        newy = pt['x']/10/2.5
        newz = pt['z']/10/2.5 
        sample_number = pt['sampleNumber']-1
        parent_number = pt['parentNumber']-1
        coordinates_axon.append([newx,newy,newz])
        if parent_number >= 0:
            edges_axon.append([parent_number,sample_number])
            # Have to deal with axon splits and terminals
            if (sample_number != parent_number + 1):
                # then the parent is at a branch split and child is at an endpoint
                atlas_id = pt['allenId']
                atlas_id_parent = axon[pt_index-1]['allenId']
                try:
                    axon_branches_dict[atlas_id_parent] +=1
                except: 
                    axon_branches_dict[atlas_id_parent] = 1
                try:
                    axon_terminals_dict[atlas_id] += 1
                except:
                    axon_terminals_dict[atlas_id] = 1

    vertex_positions_axon = np.array(coordinates_axon,dtype='<f4')
    edges_axon = np.array(edges_axon,dtype='<u4')
    
    # Encode into binary skeleton file
    filename_axon = os.path.join(precomputed_dir,str(ID_axon))
    with open(filename_axon,'wb') as outfile:
        buf = struct.pack('<II', vertex_positions_axon.shape[0], edges_axon.shape[0])
        buf += vertex_positions_axon.tobytes()
        buf += edges_axon.tobytes()
        outfile.write(buf)
    if debug:
        print("Wrote ",filename_axon)

    ### Dendrite
    ID_dendrite = ID_soma + 2
    dendrite = neuron['dendrite']
    
    coordinates_dendrite = []
    edges_dendrite = []
    dendrite_branches_dict = {}
    dendrite_terminals_dict = {}
    for pt_index,pt in enumerate(dendrite):
       # Downsample to to 25 micron / voxels and swap their axes
        newx = pt['y']/10/2.5
        newy = pt['x']/10/2.5
        newz = pt['z']/10/2.5 
        coordinates_dendrite.append([newx,newy,newz])
        sample_number = pt['sampleNumber']-1
        parent_number = pt['parentNumber']-1
        if parent_number >= 0:
            edges_dendrite.append([parent_number,sample_number])
            if (sample_number != parent_number + 1):
                # then the parent is at a branch split and child is at an endpoint
                atlas_id = pt['allenId']
                atlas_id_parent = dendrite[pt_index-1]['allenId']
                try:
                    dendrite_branches_dict[atlas_id_parent] +=1
                except: 
                    dendrite_branches_dict[atlas_id_parent] = 1
                try:
                    dendrite_terminals_dict[atlas_id] += 1
                except:
                    dendrite_terminals_dict[atlas_id] = 1

    vertex_positions_dendrite = np.array(coordinates_dendrite,dtype='<f4')
    edges_dendrite = np.array(edges_dendrite,dtype='<u4')
    
    # Encode into binary skeleton file

    filename_dendrite = os.path.join(precomputed_dir,str(ID_dendrite))
    with open(filename_dendrite,'wb') as outfile:
        buf = struct.pack('<II', vertex_positions_dendrite.shape[0], edges_dendrite.shape[0])
        buf += vertex_positions_dendrite.tobytes()
        buf += edges_dendrite.tobytes()
        outfile.write(buf)
    if debug:
        print("Wrote ",filename_dendrite)
    
    
    return True

In [7]:
def process_singledir(neuron_json_file):
    """ Function that will be run in parallel to write the skeleton files for a single neuron"""
    with open(neuron_json_file,'r') as infile:
        neuron_data = json.load(infile)
    neuron = neuron_data['neuron']
    ID_soma = json_files.index(neuron_json_file)*3
    print("ID:")
    print(ID_soma)
    return write_skeletons_singledir(neuron,ID_soma,debug=False)

In [10]:
%%time
with ProcessPoolExecutor(max_workers=12) as executor:
    for res in executor.map(process_singledir,json_files):
        try:
            print(res)
        except Exception as exc:
            print(f'generated an exception: {exc}')


ID:ID:ID:

2127


15
ID:ID:
ID:
012
ID:

ID:18


333

ID:
30
ID:ID:

246

ID:
9
ID:
36
ID:
39
ID:
42
True
ID:
45
ID:
48
ID:
51
ID:
54
ID:
57
ID:
60
ID:
63
ID:
66
ID:
69
True
True
True
True
True
True
True
True
True
True
True
ID:
72
ID:
75
True
ID:
78
ID:
81
ID:
84
True
True
True
ID:
87
ID:
90
ID:
93
True
True
True
True
ID:
96
ID:
99
True
True
ID:
102
ID:
105
True
True
ID:
108
ID:
111
ID:
114
True
ID:
117
True
ID:
120
ID:
123
True
True
ID:
126
ID:
129
True
True
True
True
ID:
132
True
ID:
135
ID:
138
True
True
ID:
141
True
ID:
144
True
ID:
147
True
ID:
150
ID:
153
True
True
ID:
156
ID:
159
ID:
162
ID:
165
ID:
168
True
True
True
True
True
ID:
171
ID:
174
True
True
ID:
177
True
ID:
180
True
ID:
183
ID:
186
True
True
ID:
189
ID:
192
ID:
195
True
True
True
ID:
198
ID:
201
True
ID:
204
True
True
ID:
207
ID:
210
True
ID:
213
True
True
ID:
216
True
ID:
219
True
ID:
222
ID:
225
ID:
228
True
True
ID:
231
ID:
234
True
True
True
ID:
237
ID:
240
True
True
ID:
243
True
ID:
246
True
ID:
249
True
ID:
25

Process ForkProcess-35:
Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/multiprocessing/queues.py", line 97, in get
    res = self._recv_bytes()
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ahoag/anaconda3/envs/djversion0p13/lib/python3.8/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/home/ahoag/anaconda3/envs/djversion0p13/li

KeyboardInterrupt: 

In [176]:
### Segment properties
# Save segment properties 
segment_props_neuron = {}
segment_props_neuron['@type'] = 'neuroglancer_segment_properties'
segment_props_neuron['inline'] = {
    'ids': [],
    'properties':[
        {
            'id':'label',
            'type':'label',
            'values': []
        }
    ]
}
skeleton_ids = [str(x) for x in range(len(json_files)*3)]
neuron_names = [os.path.basename(x).split('.')[0] for x in json_files]
skeleton_names = [name + ': ' + feature for name in neuron_names for feature in ['soma','axon','dendrite']]

segment_props_neuron['inline']['ids'] = skeleton_ids
segment_props_neuron['inline']['properties'][0]['values'] = skeleton_names 

segment_props_neuron_dir = '/jukebox/LightSheetData/neuroglancer/public/mouselight/allen_ccfv3_25um/ccfv3_25um_mouselight/segment_properties'
os.makedirs(segment_props_neuron_dir,exist_ok=True)

segment_props_neuron_filename = os.path.join(segment_props_neuron_dir,'info')
with open(segment_props_neuron_filename,'w') as outfile:
    json.dump(segment_props_neuron,outfile,indent=2)
print(f"Saved {segment_props_neuron_filename}")

# Save info file
info = {
    "@type": "neuroglancer_skeletons",
    "transform": [25000, 0.0, 0.0, 0.0, 0.0, 25000, 0.0, 0.0, 0.0, 0.0, 25000, 0.0],
    "vertex_attributes":[],
    "segment_properties":"segment_properties"
}
info_filename = '/jukebox/LightSheetData/neuroglancer/public/mouselight/all_neurons/info'
with open(info_filename,'w') as outfile:
    json.dump(info,outfile,indent=2)

print("Saved ",info_filename)

Saved /jukebox/LightSheetData/neuroglancer/public/mouselight/all_neurons/segment_properties/info
Saved  /jukebox/LightSheetData/neuroglancer/public/mouselight/all_neurons/info
